CS524: Introduction to Optimization Lecture 12
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## September 30, 2024
--------------

In [1]:
import sys
import numpy as np
import pandas as pd

from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
)

options = Options(time_limit=40,seed=42)
m = Container(options=options)

# Big assignment problem

Build some large random assignment problems and see difference in solution times

Demonstrates:
- How to set the random seed.
- How to use solver options
- How to do things in a 'sparse' way in GAMSPy
- How to do conditional processing using filters

In [2]:
def mysample(size, seed):
    return rng.uniform(low=0.0, high=100.0, size=size)

rng = np.random.default_rng(seed=options.seed)

num = 4000
i = Set(m,'horse',records=[f"h{k+1}" for k in range(num)])
j = Set(m,'jockey',records=[f"j{k+1}" for k in range(num)])

c = Parameter(m,'c',[i,j])       
c.generateRecords(density=0.15, func=mysample)

arc = Set(m,'arc',domain=[i,j])
arc.setRecords(c.records) # does all records with text as c values!

x = Variable(m,"x","positive",domain=[i,j])

# EQUATIONS #

hookup_horse = Equation(m,'hookup_horse',domain=i)  
hookup_horse[i]= Sum(j, x[i,j]) <= 1 

hookup_jockey = Equation(m,'hookup_jockey',domain=j)  
hookup_jockey[j]= Sum(i, x[i,j]) <= 1 

hookup_dense = Model(m,
    name="hookup_dense",
    equations=m.getEquations(),
    problem=Problem.LP,
    sense=Sense.MAX,
    objective= Sum([i,j], c[i,j] * x[i,j]),
)

x.up[i,j].where[~arc[i,j]] = 0.0

In [3]:
options.step_summary = True
options.write_listing_file = True
options.listing_file = "gams/hookup.lst"

hookup_dense.solve(options=options)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,398923.881931101,8001,16000001,LP,CPLEX,11.43


In [4]:
hookup_horse[i]= Sum(arc[i,j], x[i,j]) <= 1 
hookup_jockey[j]= Sum(arc[i,j], x[i,j]) <= 1 

hookup = Model(m,
    name="hookup",
    equations=[hookup_horse,hookup_jockey],
    problem=Problem.LP,
    sense=Sense.MAX,
    objective= Sum(arc[i,j], c[i,j] * x[i,j]),
)

x.l[arc] = 0
options.basis_detection_threshold = 1.0
hookup.solve(options=options)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,398923.881931101,8001,2400001,LP,CPLEX,8.108


In [6]:
results = {}

x.l[arc] = 0
hookup.solve(options=options,solver='cplex',solver_options={'lpmethod': 1, 'names': 'no'})
results['primal'] = hookup.total_solve_time

x.l[arc] = 0
hookup.solve(options=options,solver='cplex',solver_options={'lpmethod': 2, 'names': 'no'})
results['dual'] = hookup.total_solve_time

x.l[arc] = 0
hookup.solve(options=options,solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'})
results['network'] = hookup.total_solve_time

x.l[arc] = 0
hookup.solve(options=options,solver='cplex',solver_options={'lpmethod': 4, 'names': 'no'})
results['barrier'] = hookup.total_solve_time

x.l[arc] = 0
hookup.solve(options=options,solver='gurobi')
results['gurobi'] = hookup.total_solve_time

display(results)

{'primal': 40.415999852120876,
 'dual': 12.76199952699244,
 'network': 8.84599972050637,
 'barrier': 26.98900003451854,
 'gurobi': 11.937999771907926}

## Let's figure out how many horse and jockey get their top choice

In [7]:
horse_pref = Set(m,'horse_pref',domain=[i,j])
jockey_pref = Set(m,'jockey_pref',domain=[i,j])

horse_who_got_best_jockey = Set(m,'horse_who_got_best_jockey',domain=i)
jockey_who_got_best_horse = Set(m,'jockey_who_got_best_horse',domain=j)

best_jockey = Parameter(m,'best_jockey',domain=i,description='best jockey value for horse')
best_horse = Parameter(m,'best_horse',domain=j,description='best horse value for jockey')

In [8]:
best_jockey[i] = Smax(arc[i,j], c[i,j])
best_horse[j] = Smax(arc[i,j], c[i,j])

horse_pref[i,j].where[arc[i,j] & (c[i,j] == best_jockey[i])] = True
jockey_pref[i,j].where[arc[i,j] & (c[i,j] == best_horse[j])] = True

display(horse_pref.records.head)

<bound method NDFrame.head of       horse jockey element_text
0        h1  j2922             
1        h2  j3750             
2        h3  j2626             
3        h4  j3015             
4        h5  j3705             
...     ...    ...          ...
3995  h3996  j3869             
3996  h3997  j2004             
3997  h3998  j3670             
3998  h3999  j2978             
3999  h4000  j3227             

[4000 rows x 3 columns]>

In [9]:
horse_who_got_best_jockey[i] = Number(1).where[Sum(horse_pref[i,j], x.l[i,j]) > 0.01]
jockey_who_got_best_horse[j] = Number(1).where[Sum(jockey_pref[i,j], x.l[i,j]) > 0.01]

display(horse_who_got_best_jockey.records,jockey_who_got_best_horse.records)

pct_horse_happy = 100*len(horse_who_got_best_jockey.records)/num
pct_jockey_happy = 100*len(jockey_who_got_best_horse.records)/num

display(pct_horse_happy,pct_jockey_happy)

,horse,element_text
0,h2,
1,h4,
2,h5,
3,h6,
4,h10,
...,...,...
1996,h3992,
1997,h3994,
1998,h3996,
1999,h3997,


,jockey,element_text
0,j6,
1,j15,
2,j16,
3,j18,
4,j19,
...,...,...
2007,j3995,
2008,j3996,
2009,j3997,
2010,j3998,


50.025

50.3